In [45]:
!python -V
import numpy as np
import scipy as sp
import seaborn as sns

rng = np.random.default_rng(1234)

Python 3.13.3


# Question 1

## (a)

For the model

$$ dX_t = rX_t dt + \sigma X_{t}^\gamma dW_t $$

We note that the associated PDE for the corridor option price $V(t, x)$ is

$$
\begin{align*}
    &\frac{\partial V}{\partial t} + rx \frac{\partial V}{\partial x} + \frac{\sigma^2 x^{2\gamma}}{2} \frac{\partial^2 V}{\partial x^2} - rV = 0 \\
    &\\
    &V(T, x) = 𝟙_{[15, 25]} (x)
\end{align*}
$$

We recall that $V^{m}_{n} = V(m\Delta t, n \Delta x)$ and the discretizations for the implicit scheme is given by

$$
\begin{align*}
    \frac{\partial V}{\partial t} &\approx \frac{V^{m}_{n} - V^{m-1}_{n}}{\Delta t} \\
    \frac{\partial V}{\partial x} &\approx \frac{V^{m-1}_{n+1} - V^{m-1}_{n-1}}{2\Delta x} \\
    \frac{\partial^2 V}{\partial x^2} & \approx \frac{V^{m-1}_{n+1} - 2V^{m-1}_{n} + V^{m-1}_{n-1}}{(\Delta x)^2}
\end{align*}
$$

Using the fact that $x = n \Delta x$, this yields the scheme

$$
    \frac{V^{m}_{n} - V^{m-1}_{n}}{\Delta t} + rn \frac{V^{m-1}_{n+1} - V^{m-1}_{n-1}}{2} + \frac{\sigma^2 n^2 (\Delta x)^{(2\gamma - 2)}}{2} \left( V^{m-1}_{n+1} - 2V^{m-1}_{n} + V^{m-1}_{n-1} \right) - rV^{m-1}_{n} = 0
$$

We will also use the boundary condition $\frac{\partial^2 V}{\partial x^2} = 0$, which amounts to 
$$
\begin{align*}
    &V^{k}_{N+1} = 2V^{k}_{N} - V^{k}_{N-1} \\
    &V^{k}_{0} = 2V^{k}_{1} - V^{k}_{2}
\end{align*}
$$
Where $N+1, 0$ are indices beyond the $x$ scale.

Set $\alpha = \sigma^2 (\Delta x)^{2\gamma - 2}$. Then summarizing this scheme, we arrive at

$$
\begin{align*}
    V^{m}_{n} &= V^{m-1}_{n-1} \left(\frac{rn}{2} \Delta t - \frac{n^2 \alpha}{2} \Delta t \right) \\
    &+ V^{m-1}_{n} \left(1 + r \Delta t + n^2 \alpha \Delta t \right) \\
    &+ V^{m-1}_{n+1} \left(-\frac{rn}{2} \Delta t - \frac{n^2 \alpha}{2} \Delta t \right)
\end{align*}
$$

Which simplifies to the linear system $A v^{m-1} = v^m$ where $A$ is given by 

$$
    \begin{bmatrix}
        1 + 2r \Delta t & -r \Delta t & 0 & 0 & \cdots & 0 & 0 & 0 \\
        r\Delta t - 2\alpha \Delta t & 1 + r \Delta t + 4\alpha \Delta t & -r\Delta t - 2\alpha \Delta t & 0 & \cdots & 0 & 0 & 0 \\
        0 & \frac{3r}{2} \Delta t - \frac{9\alpha}{2} \Delta t & 1 + r\Delta t + 9\alpha \Delta t & -\frac{3r}{2} \Delta t - \frac{9\alpha}{2} \Delta t & \dots & 0 & 0 & 0 \\
        \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
        0 & 0 & 0 & 0 & \cdots & 0 & rN \Delta t & 1
    \end{bmatrix}
$$

Note that $n$ changes with the rows, and that the first and last row are different due to the natural boundary condition.

In [46]:
def implicit_solver(
        *,
        dt: float,
        dx: float,
        r: float,
        gamma: float,
        sigma: float,
        T: float,
        x_range: tuple[float, float]
) -> np.ndarray:
        '''
        x_range is interpreted as `[X_min, X_max]`. Necessarily, x0 should be in x_range.

        This assumes that `T / dt` and `(X_max - X_min) / dx` are integers.
        '''
        # initialize grid and initial conditions
        time_points = int(T / dt)
        price_points = int((x_range[1] - x_range[0]) / dx) + 1

        grid = np.empty((price_points, time_points))
        grid[:, -1] = np.ones(price_points)

        # find the matrix A
        # instead of constructing A explicitly,
        # i will use scipy's "solve_banded" method
        # which requires a 3 x N matrix.
        alpha = sigma**2 * (dx)**(2*gamma - 2)
        n = np.arange(1, price_points) # 1, 2, ..., N - 1

        upper_diagonal = -r * dt / 2 * n - alpha * dt / 2 * n**2
        upper_diagonal[0] = -r * dt
        upper_diagonal = np.insert(upper_diagonal, 0, 0) # for scipy

        diagonal = 1 + r * dt + alpha * dt * n**2
        diagonal[0] = 1 + 2 * r * dt
        diagonal = np.append(diagonal, 1)

        lower_diagonal = r * dt / 2 * n - alpha * dt / 2 * n**2
        lower_diagonal[-1] = r * time_points * dt
        lower_diagonal = np.append(lower_diagonal, 0) # for scipy

        A_scipy = np.vstack((upper_diagonal, diagonal, lower_diagonal))
        
        # run the scheme
        for i in range(time_points - 1, 0, -1): # M - 1, ..., 1
                grid[:, i-1] = sp.linalg.solve_banded((1, 1), A_scipy, grid[:, i])

        return grid

In [47]:
implicit_solution = implicit_solver(
    dt=0.01,
    dx=0.02,
    r=0.05,
    gamma=0.8,
    sigma=0.4,
    T=0.5,
    x_range=(15, 25)
)

In [ ]:
# graph the solution